In [51]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [68]:
df1 = pd.read_excel(r"D:\books\Copy of CSV_Matrix_2.0_Group_B_dataset(1).xlsx", sheet_name=1,skiprows=3, header=0)  
df1 = df1.drop(df1.columns[0], axis=1)
df1.columns = df1.columns.str.strip()
df1.head()


,team1,team2,match_number,venue,city,toss_winner,toss_decision,player_of_match,umpire1,umpire2,reserve_umpire,match_referee,match_type,extras,winner,looser,win_by_runs,win_by_wickets
0,Afghanistan,England,1,Narendra Modi Stadium,Ahmedabad,England,field,JE Root,HDPK Dharmasena,Nitin Menon,Sharfuddoula,J Srinath,Group,NaN,NaN,NaN,NaN,NaN
1,Sri Lanka,South Africa,2,Rajiv Gandhi International Stadium,Hyderabad,South Africa,field,KIC Asalanka,AT Holdstock,CM Brown,PR Reiffel,JJ Crowe,Group,NaN,NaN,NaN,NaN,NaN
2,Netherlands,Pakistan,3,Himachal Pradesh Cricket Association Stadium,Dharamsala,Pakistan,field,Babar Azam,JS Wilson,HDPK Dharmasena,M Erasmus,AJ Pycroft,Group,NaN,NaN,NaN,NaN,NaN
3,New Zealand,Bangladesh,4,Arun Jaitley Stadium,Delhi,Bangladesh,field,DJ Mitchell,RK Illingworth,Sharfuddoula,AG Wharf,J Srinath,Group,NaN,NaN,NaN,NaN,NaN
4,India,Australia,5,MA Chidambaram Stadium,Chennai,India,bat,GJ Maxwell,CB Gaffaney,RA Kettleborough,AT Holdstock,RB Richardson,Group,NaN,NaN,NaN,NaN,NaN


In [69]:
df2 = pd.read_excel(r"D:\books\Copy of CSV_Matrix_2.0_Group_B_dataset(1).xlsx", sheet_name=2,skiprows=3, header=0)
df2 = df2.drop(df2.columns[0], axis=1)
df2.head()

,match_id,innings,ball,batting_team,bowling_team,striker,non_striker,bowler,runs_off_bat,wides,noballs,byes,legbyes,penalty,wicket_type,player_dismissed,extras
0,1,1,0.1,Afghanistan,England,Rahmanullah Gurbaz,Ibrahim Zadran,RJW Topley,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1,0.2,Afghanistan,England,Rahmanullah Gurbaz,Ibrahim Zadran,RJW Topley,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,1,0.3,Afghanistan,England,Rahmanullah Gurbaz,Ibrahim Zadran,RJW Topley,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,1,0.4,Afghanistan,England,Ibrahim Zadran,Rahmanullah Gurbaz,RJW Topley,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,1,0.5,Afghanistan,England,Rahmanullah Gurbaz,Ibrahim Zadran,RJW Topley,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
df1.to_csv(r'D:\books\cleaned_data.csv', index=False)


In [26]:
df2.to_csv(r'D:\books\cleaned_data3.csv', index=False)

# Question 1

In [70]:
df2['extras'] = df2[['wides', 'noballs', 'byes', 'legbyes', 'penalty']].sum(axis=1)

match_extras = df2.groupby('match_id')['extras'].sum().reset_index()


In [71]:
match_extras.head()

,match_id,extras
0,1,14.0
1,2,18.0
2,3,16.0
3,4,43.0
4,5,20.0


In [72]:

df2['extras'] = df2[['wides', 'noballs', 'byes', 'legbyes', 'penalty']].sum(axis=1)


innings_runs = df2.groupby(['match_id', 'innings', 'batting_team'])[['runs_off_bat', 'extras']].sum().reset_index()
innings_runs['total_runs'] = innings_runs['runs_off_bat'] + innings_runs['extras']


innings_runs = innings_runs[['match_id', 'innings', 'batting_team', 'total_runs']]


match_results = innings_runs.sort_values(by=['match_id', 'total_runs'], ascending=[True, False])


winners_losers = match_results.groupby('match_id').apply(lambda x: pd.Series({
    'winner': x.iloc[0]['batting_team'],
    'looser': x.iloc[1]['batting_team'] if len(x) > 1 else None,
    'win_by_runs': x.iloc[0]['total_runs'] - x.iloc[1]['total_runs'] if len(x) > 1 and x.iloc[0]['innings'] == 1 else None,
    'win_by_wickets': 10 - df2[(df2['match_id'] == x.name) & (df2['innings'] == 2) & (df2['batting_team'] == x.iloc[0]['batting_team'])]['wicket_type'].count()
})).reset_index()


winners_losers['win_by_wickets'] = winners_losers.apply(lambda row: row['win_by_wickets'] if pd.isna(row['win_by_runs']) else None, axis=1)

winners_losers.head()


,match_id,winner,looser,win_by_runs,win_by_wickets
0,1,England,Afghanistan,NaN,9.0
1,2,Sri Lanka,South Africa,81.0,NaN
2,3,Pakistan,Netherlands,NaN,6.0
3,4,New Zealand,Bangladesh,102.0,NaN
4,5,Australia,India,NaN,6.0


In [73]:
match_results.head()

,match_id,innings,batting_team,total_runs
1,1,2,England,283.0
0,1,1,Afghanistan,282.0
2,2,1,Sri Lanka,286.0
3,2,2,South Africa,205.0
5,3,2,Pakistan,158.0


# Question 3

In [74]:
df1 = df1.merge(winners_losers, left_on='match_number', right_on='match_id', how='left', suffixes=('', '_new'))
df1 = df1.loc[:, ~df1.columns.duplicated()]

df1['winner'] = df1['winner_new'].fillna(df1['winner'])
df1['looser'] = df1['looser_new'].fillna(df1['looser'])
df1['win_by_runs'] = df1['win_by_runs_new'].fillna(df1['win_by_runs'])
df1['win_by_wickets'] = df1['win_by_wickets_new'].fillna(df1['win_by_wickets'])
df1.drop(columns=['winner_new', 'match_id','looser_new', 'win_by_runs_new', 'win_by_wickets_new', 'match_id_new'], inplace=True, errors='ignore')
df1.head()

,team1,team2,match_number,venue,city,toss_winner,toss_decision,player_of_match,umpire1,umpire2,reserve_umpire,match_referee,match_type,extras,winner,looser,win_by_runs,win_by_wickets
0,Afghanistan,England,1,Narendra Modi Stadium,Ahmedabad,England,field,JE Root,HDPK Dharmasena,Nitin Menon,Sharfuddoula,J Srinath,Group,NaN,England,Afghanistan,NaN,9.0
1,Sri Lanka,South Africa,2,Rajiv Gandhi International Stadium,Hyderabad,South Africa,field,KIC Asalanka,AT Holdstock,CM Brown,PR Reiffel,JJ Crowe,Group,NaN,Sri Lanka,South Africa,81.0,NaN
2,Netherlands,Pakistan,3,Himachal Pradesh Cricket Association Stadium,Dharamsala,Pakistan,field,Babar Azam,JS Wilson,HDPK Dharmasena,M Erasmus,AJ Pycroft,Group,NaN,Pakistan,Netherlands,NaN,6.0
3,New Zealand,Bangladesh,4,Arun Jaitley Stadium,Delhi,Bangladesh,field,DJ Mitchell,RK Illingworth,Sharfuddoula,AG Wharf,J Srinath,Group,NaN,New Zealand,Bangladesh,102.0,NaN
4,India,Australia,5,MA Chidambaram Stadium,Chennai,India,bat,GJ Maxwell,CB Gaffaney,RA Kettleborough,AT Holdstock,RB Richardson,Group,NaN,Australia,India,NaN,6.0


In [75]:
df1 = df1.merge(match_extras[['match_id', 'extras']], left_on='match_number', right_on='match_id', how='left', suffixes=('', '_new'))
df1['extras'] = df1['extras_new'].combine_first(df1['extras'])

df1.drop(columns=['extras_new', 'match_id'], inplace=True, errors='ignore')


df1.head()


,team1,team2,match_number,venue,city,toss_winner,toss_decision,player_of_match,umpire1,umpire2,reserve_umpire,match_referee,match_type,extras,winner,looser,win_by_runs,win_by_wickets
0,Afghanistan,England,1,Narendra Modi Stadium,Ahmedabad,England,field,JE Root,HDPK Dharmasena,Nitin Menon,Sharfuddoula,J Srinath,Group,14.0,England,Afghanistan,NaN,9.0
1,Sri Lanka,South Africa,2,Rajiv Gandhi International Stadium,Hyderabad,South Africa,field,KIC Asalanka,AT Holdstock,CM Brown,PR Reiffel,JJ Crowe,Group,18.0,Sri Lanka,South Africa,81.0,NaN
2,Netherlands,Pakistan,3,Himachal Pradesh Cricket Association Stadium,Dharamsala,Pakistan,field,Babar Azam,JS Wilson,HDPK Dharmasena,M Erasmus,AJ Pycroft,Group,16.0,Pakistan,Netherlands,NaN,6.0
3,New Zealand,Bangladesh,4,Arun Jaitley Stadium,Delhi,Bangladesh,field,DJ Mitchell,RK Illingworth,Sharfuddoula,AG Wharf,J Srinath,Group,43.0,New Zealand,Bangladesh,102.0,NaN
4,India,Australia,5,MA Chidambaram Stadium,Chennai,India,bat,GJ Maxwell,CB Gaffaney,RA Kettleborough,AT Holdstock,RB Richardson,Group,20.0,Australia,India,NaN,6.0


In [28]:
df1.to_csv(r'D:\books\mergedSheet.csv', index=False)

In [17]:
group_stage_matches = df1[df1['match_type'] == 'Group']

matches_team1 = group_stage_matches['team1'].value_counts()
matches_team2 = group_stage_matches['team2'].value_counts()


matches_played = matches_team1.add(matches_team2, fill_value=0)


matches_played_df = matches_played.reset_index()
matches_played_df.columns = ['Team', 'Matches Played']


In [18]:
matches_played_df.head()


,Team,Matches Played
0,Afghanistan,9
1,Australia,9
2,Bangladesh,9
3,England,9
4,India,9


In [19]:
matches_won = group_stage_matches['winner'].value_counts()

matches_won_df = matches_won.reset_index()
matches_won_df.columns = ['Team', 'Matches Won']

In [20]:
matches_won_df

,Team,Matches Won
0,Australia,9
1,New Zealand,7
2,India,7
3,England,6
4,Netherlands,4
5,Sri Lanka,3
6,Afghanistan,3
7,Pakistan,2
8,South Africa,2
9,Bangladesh,2


In [22]:
matches_lost = group_stage_matches['looser'].value_counts()

matches_lost_df = matches_lost.reset_index()
matches_lost_df.columns = ['Team', 'Matches Lost']
matches_lost_df

,Team,Matches Lost
0,South Africa,7
1,Bangladesh,7
2,Pakistan,7
3,Afghanistan,6
4,Sri Lanka,6
5,Netherlands,5
6,England,3
7,India,2
8,New Zealand,2


In [23]:
extras_given = pd.DataFrame()

team1_extras = group_stage_matches[['team1', 'extras']].copy()
team1_extras.columns = ['Team', 'Extras']
team1_extras['Type'] = 'Team1'


team2_extras = group_stage_matches[['team2', 'extras']].copy()
team2_extras.columns = ['Team', 'Extras']
team2_extras['Type'] = 'Team2'


extras_combined = pd.concat([team1_extras, team2_extras], ignore_index=True)


total_extras = extras_combined.groupby('Team')['Extras'].sum().reset_index()
total_extras

,Team,Extras
0,Afghanistan,189.0
1,Australia,187.0
2,Bangladesh,253.0
3,England,181.0
4,India,247.0
5,Netherlands,219.0
6,New Zealand,286.0
7,Pakistan,176.0
8,South Africa,254.0
9,Sri Lanka,242.0


In [24]:
merged_df = pd.merge(group_stage_matches, df2, left_on='match_number', right_on='match_id', how='left')

merged_df['total_runs'] = merged_df['runs_off_bat'] + merged_df['extras_x']


total_runs = pd.DataFrame()

team1_runs = merged_df.groupby('team1')['total_runs'].sum().reset_index()
team1_runs.columns = ['Team', 'Total_Runs']


team2_runs = merged_df.groupby('team2')['total_runs'].sum().reset_index()
team2_runs.columns = ['Team', 'Total_Runs']


total_runs_combined = pd.concat([team1_runs, team2_runs], ignore_index=True)


total_runs_final = total_runs_combined.groupby('Team')['Total_Runs'].sum().reset_index()


team1_extras = group_stage_matches[['team1', 'extras']].copy()
team1_extras.columns = ['Team', 'Extras']
team1_extras['Type'] = 'Team1'

team2_extras = group_stage_matches[['team2', 'extras']].copy()
team2_extras.columns = ['Team', 'Extras']
team2_extras['Type'] = 'Team2'


extras_combined = pd.concat([team1_extras, team2_extras], ignore_index=True)


total_extras = extras_combined.groupby('Team')['Extras'].sum().reset_index()


result = pd.merge(total_runs_final, total_extras, on='Team', how='left')
result['% Extras'] = (result['Extras'] / result['Total_Runs']) * 100

In [289]:
result

,Team,Total_Runs,Extras,% Extras
0,Afghanistan,104850.0,189.0,0.180258
1,Australia,102366.0,187.0,0.182678
2,Bangladesh,145240.0,253.0,0.174194
3,England,104051.0,181.0,0.173953
4,India,145971.0,247.0,0.169212
5,Netherlands,125020.0,219.0,0.175172
6,New Zealand,162296.0,286.0,0.176221
7,Pakistan,102402.0,176.0,0.171872
8,South Africa,145880.0,254.0,0.174116
9,Sri Lanka,142182.0,242.0,0.170204


In [26]:
results_list = []


for team in group_stage_matches['winner'].unique():
    matches_played = group_stage_matches[(group_stage_matches['team1'] == team) | (group_stage_matches['team2'] == team)].shape[0]
    matches_won = group_stage_matches[group_stage_matches['winner'] == team].shape[0]
    extras_given = group_stage_matches[(group_stage_matches['team1'] == team) | (group_stage_matches['team2'] == team)]['extras'].sum()
    
 
    runs_scored = group_stage_matches[group_stage_matches['winner'] == team]['win_by_runs'].sum()
    runs_conceded = group_stage_matches[group_stage_matches['looser'] == team]['win_by_runs'].sum() + group_stage_matches[group_stage_matches['looser'] == team]['extras'].sum()
    

    overs_faced = group_stage_matches[group_stage_matches['winner'] == team]['extras'].sum()
    overs_bowled = group_stage_matches[group_stage_matches['looser'] == team]['extras'].sum()


    nrr_for = runs_scored / overs_faced if overs_faced > 0 else 0
    nrr_against = runs_conceded / overs_bowled if overs_bowled > 0 else 0
    nrr = nrr_for - nrr_against
    
   
    results_list.append({
        'Team': team,
        'Matches Played': matches_played,
        'Matches Won': matches_won,
        'Extras Given': extras_given,
        'NRR For': nrr_for,
        'NRR Against': nrr_against,
        'NRR': nrr
    })

results = pd.DataFrame(results_list)
results

,Team,Matches Played,Matches Won,Extras Given,NRR For,NRR Against,NRR
0,England,9,6,181.0,4.401961,3.468354,0.933606
1,Sri Lanka,9,3,242.0,1.140845,3.081871,-1.941026
2,Pakistan,9,2,176.0,0.000000,3.869231,-3.869231
3,New Zealand,9,7,286.0,3.688073,5.132353,-1.444280
4,Australia,9,9,187.0,4.304813,0.000000,4.304813
5,Afghanistan,9,3,189.0,5.735294,4.561983,1.173311
6,Netherlands,9,4,219.0,0.669903,2.284483,-1.614580
7,India,9,7,247.0,2.076142,3.680000,-1.603858
8,South Africa,9,2,254.0,2.155172,5.127551,-2.972379
9,Bangladesh,9,2,253.0,0.000000,3.172043,-3.172043


# Question 5

In [27]:


df2 = df2.drop(columns=[df2.columns[0]])

df2['runs_off_bat'] = pd.to_numeric(df2['runs_off_bat'], errors='coerce')
df2['ball'] = pd.to_numeric(df2['ball'], errors='coerce')
df2['wides'] = pd.to_numeric(df2['wides'], errors='coerce')
df2['noballs'] = pd.to_numeric(df2['noballs'], errors='coerce')
df2['byes'] = pd.to_numeric(df2['byes'], errors='coerce')
df2['legbyes'] = pd.to_numeric(df2['legbyes'], errors='coerce')


def get_over(ball):
    try:
        return int(float(ball))
    except:
        return None

def is_third_powerplay(ball):
    over = get_over(ball)
    return 40 < over <= 50 if over is not None else False


df2['over'] = df2['ball'].apply(get_over)
df2['is_third_powerplay'] = df2['over'].apply(lambda x: 40 < x <= 50 if x is not None else False)


def calculate_powerplay_stats(df, batting_first=None):
    powerplay_data = df[df['is_third_powerplay']]
    if batting_first is not None:
        powerplay_data = powerplay_data[powerplay_data['innings'] == (1 if batting_first else 2)]
    
    team_stats = []
    for team in powerplay_data['batting_team'].unique():
        team_data = powerplay_data[powerplay_data['batting_team'] == team]
        
        
        total_runs = (team_data['runs_off_bat'].fillna(0) + 
                     team_data['wides'].fillna(0) + 
                     team_data['noballs'].fillna(0) + 
                     team_data['byes'].fillna(0) + 
                     team_data['legbyes'].fillna(0)).sum()
        
        
        legal_deliveries = len(team_data) - team_data['wides'].fillna(0).sum() - team_data['noballs'].fillna(0).sum()
        overs = legal_deliveries / 6
        
       
        wickets = team_data['wicket_type'].notna().sum()
        avg_balls_per_wicket = legal_deliveries / wickets if wickets > 0 else float('inf')
        
        team_stats.append({
            'Team': team,
            'Run Rate': round(total_runs / overs if overs > 0 else 0, 2),
            'Wickets': wickets,
            'Avg Balls per Wicket': round(avg_balls_per_wicket, 2) if avg_balls_per_wicket != float('inf') else 'N/A'
        })
    
    return pd.DataFrame(team_stats)

overall_stats = calculate_powerplay_stats(df2)
batting_first_stats = calculate_powerplay_stats(df2, batting_first=True)
batting_second_stats = calculate_powerplay_stats(df2, batting_first=False)

print("\
Overall Third Powerplay Statistics:")
print(overall_stats)
print("\
Batting First Statistics:")
print(batting_first_stats)
print("\
Batting Second Statistics:")
print(batting_second_stats)

Overall Third Powerplay Statistics:
           Team  Run Rate  Wickets  Avg Balls per Wicket
0   Afghanistan      8.85       21                 12.05
1     Sri Lanka      7.84       18                 12.89
2   New Zealand     10.64       19                 21.37
3    Bangladesh      5.16       17                 16.00
4         India      8.01       22                 16.68
5     Australia      8.55       19                 19.47
6       England      8.43       23                 14.09
7  South Africa      6.73       18                 12.33
8      Pakistan      5.76       18                 15.22
9   Netherlands      7.72       13                 22.23
Batting First Statistics:
           Team  Run Rate  Wickets  Avg Balls per Wicket
0   Afghanistan      9.09       18                 11.67
1     Sri Lanka      7.35       12                 12.25
2   New Zealand     11.87       16                 19.88
3         India      8.45       21                 12.38
4       England      8.50 